# Queries and filters
There is not much sense in having a database if you don't know how to get data from it. Let's find out how it works in Django!

Let's talk about the Model object manager. We will use it to get and filter the data for a particular model. Once you learn the syntax rules, you'll be able to easily make queries to your database. It gives you the flexibility to retrieve any objects you want.

Reading the data is the most common operation for a web application. The clients get data from the server more often than modify or delete it.


# Model object manager
An instance of the Model class represents a single row in the table of your database. To start working with a set of rows, call the Model object manager methods.

The manager is a special class to get object(s) from a database to modify them. Django model manager is the interface through which database query operations are provided to Django models

Currently, we work on the tournament application for the Quidditch league. The season is coming, but the website is not ready! Wizards from Hogwarts get used to working with the books and papers, so they know nothing on databases. Fortunately, you don't need magic to start querying and searching. We create models Team and Player and that's how we define

In [ ]:
from django.db import models


class Team(models.Model):
    name = models.CharField(max_length=64)


class Player(models.Model):
    height = models.FloatField()
    name = models.CharField(max_length=64)
    team = models.ForeignKey(Team, on_delete=models.CASCADE)


team_model_manager = Team.objects
player_model_manager = Player.objects

It's not necessary to give an alias name to the Manager, you can use its methods simply like this: Team.objects.filter(name="Ballycastle Bats"). You can choose what you like more, but for clarity, we will access it directly in all the examples.

In [ ]:
falmouth_falcons = Team.objects.create(name="Falmouth Falcons")
montrose_magpies = Team.objects.create(name="Montrose Magpies")

Player.objects.create(name="Karl Broadmoor", height=180, team=falmouth_falcons)
Player.objects.create(name="Kevin Broadmoor", height=183, team=falmouth_falcons)
Player.objects.create(name="Alasdair Maddock", height=175, team=montrose_magpies)
Player.objects.create(name="Lennox Campbell", height=197, team=montrose_magpies)

You can also use bulk_create method to add a list of objects into the database:



In [ ]:
 Player.objects.bulk_create([
    Player(name="Karl Broadmoor", height=180, team=falmouth_falcons),
    Player(name="Kevin Broadmoor", height=183, team=falmouth_falcons),
    Player(name="Alasdair Maddock", height=175, team=montrose_magpies),
    Player(name="Lennox Campbell", height=197, team=montrose_magpies)
])

You can use the code above in any file of your app, but don't forget to import model classes from models.py and remember that you should migrate your models before using them.

# Getting an object
One step at a time, we will start from getting the team we want and then move on to getting a distinct player.
- We will carefully pick the parameters for our first query. Our Team model has two fields: id and name. The id field is generated automatically for every model, though we do not specify it explicitly.

We are certain that we have a team named Falmouth Falcons. Let's try to get it with the manager:


In [ ]:
falcons = Team.objects.get(name="Falmouth Falcons")

Looks fine. But what happens if we get a team that doesn't exist?



In [ ]:
tornados = Team.objects.get(name="Tutshill Tornados")

This call raises the Team.DoesNotExist exception. Unlike Python's dict get method, the manager's get method may raise an Exception. To prevent this situation and keep our program from crashing, you can wrap this call in a try-except construction:

In [ ]:
try:
    tornados = Team.objects.get(name="Tutshill Tornados")
except Team.DoesNotExist:
    ...

Let's try to get the Karl Broadmoor player profile from the database:

In [ ]:
karl_broadmoor = Player.objects.get(name="Karl Broadmoor")

Karl plays for Falmouth Falcons, so we get his profile with no errors. Suppose you want to make a query that returns multiple objects:

In [ ]:
falcons = Team.objects.get(name="Falmouth Falcons")
falcon_player = Player.objects.get(team=falcons)

You will get not a player but a Player.MultipleObjectsReturned exception.

So, to prevent exceptions, you should keep in mind two rules:

You can only pass the parameters with the names of the fields of your model or with valid field lookups;

You should be sure that with this query you will get exactly one object.

# Filtering objects


In [ ]:
tornados = Team.objects.filter(name="Tutshill Tornados")

Filtering objects only by their exact value is not always convenient: you may want to get objects that satisfy a trickier condition. For example, it could be games where the home team scored more than 12 points. We'll start with this query:

In [ ]:
great_score_at_home_games = Game.objects.filter(home_team_points__gt=12)

The special syntax for the parameter is: the field name, double underscores, the field lookup, a special name for actions on the field value you want to make when filtering data.

To retrieve an object from the QuerySet you can iterate it over or get the item by the index as you get it from the Python's list.

In [ ]:
twentieth_century_games = Game.objects.filter(date__year__range=(1900, 1999))

In [ ]:
tornados = Team.objects.filter(name="Tutshill Tornados")
if len(tornados) == 1:
    tornados_team = tornados[0]

In [ ]:
class Player(models.Model):
    name = models.CharField(max_length=64, null=True, default=None)

noname_player = Player.objects.create()
players_without_name = Player.objects.filter(name__isnull=True)

In [ ]:
twentieth_century_games = Game.objects.filter(date__year__gte=1900, date__year__lte=1999)

In [ ]:
from datetime import date

twentieth_century_games = Game.objects.filter(
    date__gte=date(1900, 1, 1), date__lte=date(1999, 12, 31)
)

In [ ]:
Team.objects.filter(name="Tutshill Tornados").count()

In addition, we want to get a Falmouth Falcons player. Let's do it with the combination of the filter and first methods:

In [ ]:
falcons = Team.objects.get(name="Falmouth Falcons")
falcon_player = Player.objects.filter(team=falcons).first()

# Using get_or_create
While working with databases, you'll often encounter situations where you want to get a specific object if it exists, or create it if it doesn't. Django provides a handy method for this - get_or_create.

This method tries to fetch an object from your database based on the parameters you provide. If the object exists, it returns a tuple containing the object and a boolean value False. If the object does not exist, this method creates it with the provided parameters and returns a tuple containing the new object and True.

Let's take an example. Suppose we want to get a team with the name "Puddlemere United". If it exists, we want to get it, otherwise, we want to create it. Here's how we can do it:

In [ ]:
team, created = Team.objects.get_or_create(name="Puddlemere United")

In this case, if a team named "Puddlemere United" exists in the database, team will be that Team instance and created will be False. If the team does not exist, a new team will be created, team will be the new Team instance, and created will be True.

There's another way to achieve the same result, although it involves more steps. First, you check if the object exists using the .exists() method, and if it doesn't, you create it. Here's how you can do it:

In [ ]:
if not Team.objects.filter(name="Puddlemere United").exists():
    team = Team.objects.create(name="Puddlemere United")
    created = True
else:
    team = Team.objects.get(name="Puddlemere United")
    created = False

In this case, if a team named "Puddlemere United" exists in the database, team will be that Team instance and created will be False. If the team does not exist, a new team will be created, team will be the new Team instance, and created will be True.

# Collections for lookups
```python

special_home_games = Game.objects.exclude(home_team=tutshill_tornados) \
                                 .exclude(home_team=appleby_arrows)


special_home_games = Game.objects.filter(home_team__in=[falmouth_falcons, montrose_magpies])       

## Field lookups for foreign keys
By now, you've learned how to filter data and use it for another query. First, we store falmouth_falcons as a variable. Then we can filter the games where Falcons were the home team. But do we need to store it in a variable to make a query? The answer is no, as we can directly access fields of foreign keys through lookups.
```python
falcons_home_games = Game.objects.filter(home_team__name="Falmouth Falcons")
```
- Use double underscores again. Now, the structure consists of a foreign key field name, double underscores, and a foreign key model field.

- To delve deeper, let's combine the access to the field of the foreign model with field lookups:
```python
falcons_home_games = Game.objects.filter(home_team__name__contains="Falcons")
```
- The use of double underscores is akin to access through a period, with fields resembling class attributes and lookups mirroring class methods. Hence, you can combine them just like you would combine accesses using a period.

We can do the same with many-to-many relationships. For example, let's consider the Stadium model, which contains information about its name and the teams that have played in it:
```python
from django.db import models

class Stadium(models.Model):
    name = models.CharField(max_length=64)
    team = models.ManyToManyField(Team)
```
Now we can retrieve all the stadiums where Falmouth Falcons have played:
```python
falmouth_falcons_stadiums = Stadium.objects.filter(team__name="Falmouth Falcons")

# Nested queries

Let's say you want to find all teams that have scored over 100 runs in home games. To accomplish this task, we can use nested queries.


In [ ]:
from django.db.models import Subquery

# define a subquery
subquery = Game.objects.filter(home_team_points__gt=100).values('home_team')

# use a subquery in the main query
high_scoring_teams = Team.objects.filter(id__in=Subquery(subquery)).values('name')

In this example, the inner query (or subquery) selects home_team from the Game model where home_team_points is greater than 100. The outer query then selects name from the Team model, where id values match those returned by the subquery.

Nested queries in Django can be useful in many situations. Here are some scenarios where you might employ nested queries:

- Data Filtering: Subqueries can filter data in the main query based on the subquery's results. This can be useful when you need to retrieve data meeting certain criteria that a simple filter cannot express.

- Data Aggregation: Nested queries can aggregate data. For example, you can count the number of records matching a certain condition using a subquery, and then incorporate that information into your main query.

- Performance Optimization: In some cases, nested queries can be more efficient than running multiple separate queries. However, this efficiency depends on your specific situation, so it's vital to monitor your queries' performance.

# Get or 404

It can often happen that the object we want to get does not exist. Such requests also need proper handling, and one method is using the ```get_object_or_404``` function. This convenient Django function attempts to retrieve an object with the provided parameters. However, if the targeted object is not found, it raises a 404 Not Found error, leading to a 404 error page being displayed to the user.





In [ ]:
from django.shortcuts import get_object_or_404
from .models import Team, Game

def game_detail(request, game_id):
    game = get_object_or_404(Game, id=game_id)
    return render(request, 'game_detail.html', {'game': game})

In this example, get_object_or_404 is used to get a game with a specific id. If no game with that id is found, Django automatically returns a 404 error page.

This function is useful when you want to display the details of a specific object, such as a game. If the object does not exist, the user is automatically directed to a 404 error page, which indicates proper error management.

Remember that get_object_or_404 is just a convenient wrapper around Model.objects.get(). While it offers no additional functionality compared to a regular get() request, it does enhance your code's readability and cleanliness.

# Filter Method

You have a collection of iPhones, with each having unique features. Django, in this case, is your expert organizer, and the filter() method is your tool to pick out what you're looking for. You want to find a particular type of iPhone with 256 GB of storage. The filter() method in Django does just that! But first, check the model:


```python 
from django.db import models

class Iphone(models.Model):
    storage = models.IntegerField()
    color = models.CharField(max_length=50)
    # Another fields

```
#### You can narrow down your collection and show only those iPhones that meet your criteria.

```python
from myapp.models import Iphone

# Simple filter: get objects with "storage" equal to 256
result = Iphone.objects.filter(storage=256)

You want to enrich the criteria: a collection of iPhones with 256 GB storage or those with a price of more than 1,000. We can't do that, as the filter() functionality does not allow you to create complex queries with different conditions. That's why Django has a tool called Q object. It is a utility that enables you to create queries with combined logical operators: and(&), or(|), or negation(~). With Q, you can create any filter. To use it, import the Q class from the django.db.models module:

In [ ]:
# Complex filter: get objects with price greater than 1000 or color is black
from django.db.models import Q

complex_query = Q(price__gt=1000) | Q(color='black')
filtered_iphones = Iphone.objects.filter(complex_query)

If you understand how Q objects work, you can create any filter. You can check all the options: for example, we need an iPhone X or 11 with a 4+ rating and a price lower than 1000:

In [ ]:
filtered_iphones = Iphone.objects.filter(
    (Q(model='X') | Q(model='11')) & Q(rating__gt=4) & ~Q(price__gt=1000)
)

# Mix models data
In an actual project, you will encounter the so-called nested queries. They involve embedding one query in another, which allows you to retrieve data based on conditions derived from the results of another query. This technique allows you to increase the level of complexity of your data retrieval strategy. Suppose you have iPhone and User models and want to find users with at least one iPhone of a particular model.

In [ ]:
from django.db.models import Q

from myapp.models import Iphone, User


# Nested query without Q
users_with_iphone_x = User.objects.filter(
    id__in=iPhone.objects.filter(
        model='iPhone X'
    ).values('owner_id')
)


# Nested query with Q
q_object = Q(model='iPhone X')
users_with_iphone_x = User.objects.filter(
    id__in=iPhone.objects.filter(q_object).values('owner_id')
)

# Referencing and updating fields

```python
# Suppose you have a blog post Django model.

# Importing the models class
from django.db import models
from django.contrib.auth.models import User # Importing the user model

# The blog post model
class BlogPost(models.Model):
    title = models.CharField(max_length=250)
    text = models.TextField()
    likes = models.PositiveIntegerField(default=0)
    views = models.PositiveIntegerField(default=0)
    comment_count = models.PositiveIntegerField(default=0)
    author = models.ForeignKey(User, on_delete=models.CASCADE) # Refers to the user model
    published_date = models.DateTimeField(auto_now_add=True)  # Automatically set on creation
    last_modified = models.DateTimeField(auto_now=True)

    def __str__(self):
        return self.title


Working with this model, you can modify a field in the database after migrations. Say you want to increment the likes field by one, you would write this code.

In [ ]:
# Importing the post model
from app.models import BlogPost

# Assuming post id is 1
post_id = 1

# Importing the object to python
post = BlogPost.objects.filter(id=post_id)

# Increment the likes field
post.likes += 1

# Save the post object after updating the likes field
post.save()

The code above involves importing the object to Python, updating a property, and re-saving. A much more efficient way of doing this is with the Django F object.

In [ ]:
from app.models import BlogPost
from django.db.models import F  #Importing the F object

# Assuming post id is 1
post_id = 1

# Updating the post in database directly
BlogPost.objects.filter(id=post_id).update(likes=F('likes') + 1)

# F objects in filters
You can also use the F object in filters to compare the values of different fields in a model. This allows the creation of more complex and dynamic queries from a database level. The ORM provides various lookup types that you can use in queries. Examples are exact which checks if values are equal, 
- ```gt``` checks if a value is greater than another 
- ```gte``` checks if a value is greater than or equal to another
-  ```lte``` checks if a value is less than or equal to another.


In analytics and database management, it's common practice to use field combinations or their interrelation for insightful analysis. Django's F() objects offer a flexible way to dynamically query databases based on these relationships. Here is a scenario where such queries prove invaluable in deriving meaningful analytics.

In [ ]:
# Query to use filter posts with likes equal to views
posts = BlogPost.objects.filter(likes__exact=F('comment_count'))

The query assesses user engagement by comparing the number of comments to the number of likes on a post. Here is another example that looks for items that need to be restocked in a grocery store by comparing the in_stock field with reorder_quantity field.



In [ ]:
# Fetch products that need restocking based on stock levels and reorder quantity
low_stock_items = Product.objects.filter(stock__lt=F('reorder_quantity'))

The Django F object isn't just limited to integer-type fields when writing queries. Here is an example, a query that retrieves posts with views greater than or equal to the length of their title.

In [ ]:
from django.db.models.functions import Length  # Importing the Length function

posts = BlogPost.objects.filter(
    views__gte=Length(F('title'))
)

You can also use the F object to query using date-time fields. The example below queries for posts which were published three days after they were modified.

In [ ]:
from datetime import timedelta

# Fetch posts where the published_date is greater than last_modified by 3 days
posts = BlogPost.objects.filter(
    published_date__gt=F('last_modified') + timedelta(days=3)
)

# Dynamic fields and annotations with f expressions

Dynamic fields in Django are fields created or modified on the fly during the process of executing queries. These fields are not stored in the database. Since creating them essentially involves instructing Django to calculate or generate new values based on existing fields, this is all done in memory.

Annotations allow you to add extra information to each object in a query set. Annotations are often used to include calculated values or other derived information in the results of a database query. The F object in Django allows you to reference the value of a field in a model when writing a database query.

Here is an example using the blog post model from before.



In [ ]:
# Query to annotate likes-to-views ratio to a dynamic field
posts = BlogPost.objects.annotate(likes_to_view_ratio=F('likes') / F('views'))

This example retrieves all posts and creates a dynamic field, which is the ratio of likes to view. The field is now created for all the post objects the query returns. We can then pass the results of the query to a template or use it. Here is an example of how the dynamic field can be used.

In [ ]:
# Looping throgh the posts query set
for post in posts:
   print(f"{post.title} - Likes-to-view ratio: {post.likes_to_view_ratio}")

You can also calculate aggregates and assign the values to a dynamic field. Dynamic fields and annotations prove powerful with practical examples, such as generating reports or populating leaderboards, which illustrate their real-world utility. Imagine a scenario where the likes-to-views ratio is utilized to showcase popular posts on a website.

If the fields you are combining are of different types, you will need to tell Django what kind of field will be returned. You will need to wrap the expression with an ExpressionWrapper since the F object does not support output_field which is used to define the expected data type of value returned from a query.

Say we have a model for an event.

In [ ]:
# Import the model class
from django.db import models

# The event model
class Event(models.Model):
    name = models.CharField(max_length=200)
    starts_at = models.DateTimeField()
    duration = models.DurationField()

    def __str__(self):
        return self.name

If you were to query the model for events and annotate a field called ended_at which is a field that is the sum of starts_at and duration, this is how you would go about it.

In [ ]:
from app.models import Event
from django.db.models import F, ExpressionWrapper, DateTimeField

# Using event wrapper to designate the FieldType to be returned
events = Event.objects.annotate(
    ExpressionWrapper(
        ended_at=F('starts_at') + F('duration'), output_field=DateTimeField())
    )

This way, Django knows that it should return ended_at as a DateTimeField which is the field type required for this field. Dynamic fields like this can then be seamlessly passed to the template, where it can be displayed.

```html
<ul>
    {% for event in events %}
        <li>{{ event.ended_at }}</li>
    {% endfor %}
</ul>

# Logical operations on f expressions
The F object also allows you to use logical operations when writing queries to a database. These logical operands you can use include AND(&), OR(|) and NOT(~).

Here is an example that utilizes the blog post model from before to query for posts with both likes and views greater than the sum of the comment_count field and a hundred without the F object.

In [ ]:
# Assuming you have imported the post model
# Import F and Q objects
from django.db.models import F, Q

# Normal way to query information
annotated_data = BlogPost.objects.annotate(total=F('comment_count') + 100)
posts = annotated_data.filter(likes__gt=F('total'), views__gt=F('total'))

Now, here is the same query using the F object and the AND(&) operand. The example annotates a field called total_fields which is the sum of likes and views for every record. It then filters the data for posts with both likes and views greater than total_fields.

Note, the query does use the Q object as well.

In [ ]:
# Assuming you have imported the post model
# Importing F and Q objects
from django.db.models import F, Q

# Querying using logical operations
annotated_data = BlogPost.objects.annotate(total=F('comment_count') + 100)
posts = annotated_data.filter(Q(likes__gt=F('total')) & Q(views__gt=F('total')))

We can also implement the same query using the OR(|) operand. This way, the query set will contain posts which satisfy at least one of the conditions in the query.

In [ ]:
# Assuming you have imported the post model
# Importing F and Q objects
from django.db.models import F, Q

# Querying using logical operations
annotated_data = BlogPost.objects.annotate(total=F('comment_count') + 100).
posts = annotated_data.filter(Q(likes__gt=F('total')) | Q(views__gt=F('total')))

And here is an example that utilizes the NOT(~) operand. The posts in the query set have the condition for the query as false. This example annotates the field total the same way as before and excludes all posts with likes equal to total.



In [ ]:
# Assuming you have imported the post model
# Importing the F and Q objects
from django.db.models import F, Q

# Function to negate condition
def search_post(request):
    annotated_data = BlogPost.objects.annotate(total=F('comment_count') + 100)
    posts = annotated_data.filter(~Q(total=F('likes')))

Logical operations have the advantage of allowing a programmer the flexibility to construct more complex queries with multiple conditionals at once. Here is an example of how to query blog posts with views not equal to total or total greater than the number of likes.

In [ ]:
# Assuming that you have imported the post models
# Importing the F and Q objects
from django.db.models import F, Q

# Query for posts
def search_post(request):
    annotated_data = BlogPost.objects.annotate(total=F('comment_count') + 100)
    posts = annotated_data.filter(~Q(total=F('views')) | Q(total__gt=F('likes'))

When using Django's F and Q object with logical operands, knowing the order of operations is key. It's similar to math class, when in doubt, use brackets. Here's why:

- NOT: Comes first, flipping a condition
- AND: Second in line, demanding both conditions to be true
- OR: The last to be evaluated, allowing even one condition to be true.

When creating complex queries, using brackets helps maintain clarity.

Also, breaking down parts of your expression into separate variables makes the final filter argument more readable and easier to understand. Here is an example.

In [ ]:
from django.db.models import F, Q

# Break down the complex filter into smaller parts
likes_condition = Q(likes__gt=F('total'))
views_condition = Q(views__gt=F('total'))

# Combine these conditions using OR operand
combined_condition = likes_condition | views_condition

# Now use this combined condition in your filter
annotated_data = BlogPost.objects.annotate(total=F('comment_count') + 100)
posts = annotated_data.filter(combined_condition)

# Func expressions
Functional expressions, often referred to as Func expressions, offer a robust way to enhance database performance by leveraging SQL functions like COALESCE, LOWER, and SUM directly within the database.

The LOWER function is used to convert a string to lowercase. Here is a query that retrieves blog posts from the blog post model and annotate a field called title_lower whose value is going to be the lowercase of the post's title, here is how you would go about it.

In [ ]:
# Assuming you have imported the post model
# Importing necessary classes
from django.db.models import Func, F

# Querying database and annotating field
posts = BlogPost.objects.all()
posts.annotate(title_lower=Func(F('title'), function="LOWER"))

The SUM function is used to calculate the sum of values in a column in a database table. Here is an example that uses the function to compute the sum of likes and assigns it to a dynamic field called total_likes.

In [ ]:
from django.db.models import Func, F

# Annotating the queryset with the total sum of 'likes' using Func expression
posts_with_likes_sum = BlogPost.objects.annotate(
    total_likes=Func(F('likes'), function='SUM')
)

COALESCE, although less commonly used, is versatile and can handle NULL values within SQL queries. Say we want to categorize the blog post likes into three categories: 'LOW' if a post has likes that are less than 30, 'MID' if it has likes between 30 and 80, and 'HIGH' if likes are greater than 30, based on certain thresholds. If the likes field is NULL, the the value of like_label becomes "UNDEFINED". We also set the output_field to a character field. Here is how we would go about it.

In [ ]:
from django.db.models import Func, Value, CharField
from django.db.models.functions import Coalesce, Case, When

# Annotating the queryset with text labels based on 'likes' thresholds using COALESCE
posts_with_like_labels = BlogPost.objects.annotate(
    like_label=Coalesce(
        Case(
            When(likes__lt=30, then=Value('LOW')),
            When(likes__range=[30, 80], then=Value('MID')),
            When(likes__gt=80, then=Value('HIGH')),
            output_field=CharField(),
        ),
        Value('UNDEFINED'),  # Default label for other cases (optional)
    )
)

You should consider employing Func expressions when:

When you need to utilize specific SQL functions like COALESCE, LOWER, SUM, or other custom functions.
When dealing with NULL values thus ensuring data consistency.
Note, Django ORM enebles the usage of Func expressions across various database systems and is thus:

Database independent
Cross database compatibility
Optimal portability since very little code change is required when changing database engine

Func expressions also form the building blocks for other database functions. They allow the programmer to write custom database functions to suit the task at hand. Here is an example of the same example we used to categorize posts based on the number of likes but with a custom database function written using func expressions.

In [ ]:
from django.db.models import Func, F, Value, CharField

# Define a custom Func expression to categorize likes into text labels
class LikesLabel(Func):
    function = 'CASE'
    template = "CASE WHEN %s < 30 THEN %s WHEN %s < 80 THEN %s ELSE %s END"
    output_field = CharField()

# Annotate the queryset with the likes labels using the custom Func expression
posts_with_like_labels = BlogPost.objects.annotate(
    like_label=LikesLabel(
        F('likes'), Value('LOW'), F('likes'), Value('MID'), Value('HIGH')
    )
)